# HSBC jobs posting data scrapper

## Descripción del problema

Se necesita obtener todas las ofertas laborales abiertas del banco HSBC, detallando:

- Locación del puesto
- Sector
- Tipo de jornadad
- Modalidad de trabajo
- Fecha de inicio de la búsqueda
- Fecha de finalización de la búsqueda
- Link a la descripción de la posición

## Obtención de datos

Utilizando python, herramientas de web scrapping: requests, BeautifulSoup y herramientas de manejo de datos: pandas, podemos utilizar la url publica de ofertas de trabajo de HSBC: https://mycareer.hsbc.com/en_GB/external/SearchJobs/ y obtener los datos que necesitamos y descargarlos a un archivo CSV para luego ser analizados.

### Estructura del portal de empleos

El portal de empleos de HSBC de acceso público por la URL https://mycareer.hsbc.com/en_GB/external/SearchJobs/ se organiza de la siguiente manera:
- Cada página muestra hasta 50 resultados, modificable utilizando la query **pipelineRecordsPerPage** url.
- Los resultados mostrados en la página dependen del valor **pipelineOffset** en la query url.
- Una pagina sin resultados muestra el mensaje _"There are no jobs that match your search criteria. Please amend your filters and try again"_

Conociendo esta estructura y comportamiento se puede utilizar la siguiente estrategia de scrapping.

### Estrategia de scrapping

#### Obtención de jobs posting

1. Hacer un requests.get a la url del sitio, modificando los query params, hasta encontrar el mensaje de error mencionado.
2. Para cada resultado de las requests.get, obtener todos los html tag article inlcuidos en el div con class=section__content__results
3. En una lista_total articulos, guardar los articulos obtenidos de cada requests.get

#### Extracción de datos de articulos

Una vez obtenidos todos los articulos del paso anterior, extraer:
1. Titulo del puesto
2. Locación
3. Sector
4. Horario
5. Modalidad de trabajo
6. Fechas
7. Link a la descripción

Ver [funciones de extracción](#Funciones-de-extracción)

#### Presentación de datos

Una vez extraídos los datos de los articulos, generar un pandas DataFrame y exportar a un archivo CSV.

[Descargar datos](./hsbc_jobs_data.csv)

## Implementación

In [1]:
import requests as re
from bs4 import BeautifulSoup
import pandas as pd

### Extracción de artículos

In [2]:
RECORDS_PER_PAGE = 50
get_url = lambda offset: f"https://mycareer.hsbc.com/en_GB/external/SearchJobs/?pipelineRecordsPerPage={RECORDS_PER_PAGE}&pipelineOffset={offset}"

In [3]:
error_message = "There are no jobs that match your search criteria. Please amend your filters and try again"
end_of_articles = lambda articles: articles[0].h3.text.strip(" \n\r") == error_message

In [4]:
def get_articles(offset):
    r = re.get(get_url(offset))
    soup = BeautifulSoup(r.text, "html.parser")
    tiles = soup.find("div", {"class": "section__content__results"})
    articles = tiles.find_all("article")
    return articles

In [5]:
total_articles = []
offset = 0
articles = get_articles(offset)
print("Postings obtenidos: ")
while not end_of_articles(articles):
    print(offset, end=" ")
    total_articles += articles
    offset += 50
    articles = get_articles(offset)

Postings obtenidos: 
0 50 100 150 200 250 300 350 400 450 500 550 600 650 700 750 800 850 900 950 1000 1050 1100 1150 1200 1250 1300 1350 1400 1450 1500 1550 1600 1650 1700 1750 1800 1850 1900 1950 2000 2050 2100 2150 2200 2250 2300 2350 2400 2450 2500 2550 2600 2650 2700 2750 2800 2850 2900 2950 3000 3050 3100 3150 3200 3250 3300 3350 3400 3450 3500 

### Funciones de extracción

In [6]:
# Job Post items container
get_data_container = lambda article: article.find(attrs={"class": "article__header__text__subtitle"}).find_all(attrs={"class": "item__container"})

In [7]:
# Extracting functions
get_article_title = lambda article: article.find("h3").find("a").string.strip(" \n\r")
get_article_location = lambda container: container[0].find(attrs={"class": "location"}).string.strip(" \n\r")
get_article_sector = lambda container: container[1].find("span").string.strip(" \n\r")
get_article_hours = lambda container: container[2].find("span").string.strip(" \n\r")
get_article_worktype = lambda container: container[3].find("span").string.strip(" \n\r")
get_article_dates = lambda container: [x.strip(" \n\r") for x in container[4].find("span").string.replace(" ", "").split("\n\n-")]
get_job_link = lambda article: article.find(attrs={"class": "article__header__actions"}).a["href"]

### Extracción de datos de artículos

In [23]:
def get_data_from_articles(articles):
    data = []
    for a in articles:
        container = get_data_container(a)
        title = get_article_title(a)
        location = get_article_location(container)
        sector = get_article_sector(container)
        hours = get_article_hours(container)
        worktype = get_article_worktype(container)
        dates = get_article_dates(container)
        link = get_job_link(a)
        data.append({
            "title": title,
            "location": location,
            "sector": sector,
            "hours": hours,
            "worktype": worktype,
            "dates": dates,
            "link": link,
        })
    return data

In [24]:
df = pd.DataFrame(get_data_from_articles(total_articles[:-1]))

In [25]:
df["from_date"] = df["dates"].str[0]
df["to_date"] = df["dates"].str[1]

In [26]:
df.drop(columns=["dates"], inplace=True)

In [27]:
df

,title,location,sector,hours,worktype,link,from_date,to_date
0,WPB IT Audit Manager,"Mumbai, India",Audit,Permanent - Full Time,Office Working,https://mycareer.hsbc.com/en_GB/external/Pipel...,21-Sep-2022,04-Oct-2022
1,Analyst,"Mumbai, India","Investment Banking, Markets and Research",Permanent - Full Time,Office Working,https://mycareer.hsbc.com/en_GB/external/Pipel...,21-Sep-2022,05-Oct-2022
2,CSEM Analyst,"Azcapotzalco, Mexico",Operations,Permanent - Full Time,Hybrid Working,https://mycareer.hsbc.com/en_GB/external/Pipel...,21-Sep-2022,05-Oct-2022
3,Trainee Software Engineer/Wealth & Personal Ba...,"Pune, India",Technology,Permanent - Full Time,Office Working,https://mycareer.hsbc.com/en_GB/external/Pipel...,21-Sep-2022,31-Dec-2025
4,Lead Consultant Specialist,"Guangzhou, Mainland China",Technology,Fixed Term - Full Time,Office Working,https://mycareer.hsbc.com/en_GB/external/Pipel...,21-Sep-2022,31-Dec-2025
...,...,...,...,...,...,...,...,...
3502,Personal Banker (Type 1) RBWM,"COURBEVOIE, France",Branch and Retail Banking,Permanent - Full Time,Office Working,https://mycareer.hsbc.com/en_GB/external/Pipel...,05-Dec-2021,30-Sep-2022
3503,汇丰私人财富规划北京分区总经理Field Manager - HSBC Personal W...,"Beijing, Mainland China",Insurance,Fixed Term - Full Time,Office Working,https://mycareer.hsbc.com/en_GB/external/Pipel...,01-Dec-2021,31-Dec-2025
3504,"Customer Service Officer, Securities and Retai...","Kowloon Bay, Hong Kong",Call Centre,Permanent - Full Time,Office Working,https://mycareer.hsbc.com/en_GB/external/Pipel...,18-Nov-2021,31-Dec-2025
3505,Jade Relationship Manager RBWM,"Paris, France",Branch and Retail Banking,Permanent - Full Time,Office Working,https://mycareer.hsbc.com/en_GB/external/Pipel...,28-Sep-2021,30-Sep-2022


### Export de resultados

In [28]:
df.to_csv("hsbc_jobs_data.csv")